In [151]:
import mlflow
from prettytable import PrettyTable
import tempfile
import os
import shutil
from mlflow.utils.autologging_utils import try_mlflow_log

In [152]:
mlflow.tracking.set_tracking_uri("http://localhost:5000/")
mlflow.start_run(run_name="CaptumExample")

<ActiveRun: >

In [153]:
import spacy

import torch
import torchtext
import torchtext.data
import torch.nn as nn
import torch.nn.functional as F

from torchtext.vocab import Vocab

from captum.attr import LayerIntegratedGradients, TokenReferenceBase, visualization

nlp = spacy.load('en')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [154]:
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
                
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)

        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [sent len, batch size]
        
        #text = text.permute(1, 0)
                
        #text = [batch size, sent len]
        
        embedded = self.embedding(text)

        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
            
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
                
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))

        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

In [155]:
! pip install prettytable

In [156]:
def count_model_parameters(model):
        table = PrettyTable(["Modules", "Parameters"])
        total_params = 0

        for name, parameter in model.named_parameters():

            if not parameter.requires_grad:
                continue

            param = parameter.nonzero(as_tuple=False).size(0)
            table.add_row([name, param])
            total_params += param
        print(table)
        print(f"Total Trainable Params: {total_params}")
        return table, total_params

In [157]:
count_model_parameters(model)

+------------------+------------+
|     Modules      | Parameters |
+------------------+------------+
| embedding.weight |  5099050   |
|  convs.0.weight  |   15000    |
|   convs.0.bias   |    100     |
|  convs.1.weight  |   20000    |
|   convs.1.bias   |    100     |
|  convs.2.weight  |   25000    |
|   convs.2.bias   |    100     |
|    fc.weight     |    300     |
|     fc.bias      |     1      |
+------------------+------------+
Total Trainable Params: 5159651


(<prettytable.PrettyTable at 0x7f82abf785b0>, 5159651)

In [158]:
summary, params = count_model_parameters(model)

tempdir = tempfile.mkdtemp()
try:
    summary_file = os.path.join(tempdir, "model_summary.txt")
    with open(summary_file, "w") as f:
        f.write(str(summary))

    try_mlflow_log(mlflow.log_artifact, local_path=summary_file)
finally:
    shutil.rmtree(tempdir)

+------------------+------------+
|     Modules      | Parameters |
+------------------+------------+
| embedding.weight |  5099050   |
|  convs.0.weight  |   15000    |
|   convs.0.bias   |    100     |
|  convs.1.weight  |   20000    |
|   convs.1.bias   |    100     |
|  convs.2.weight  |   25000    |
|   convs.2.bias   |    100     |
|    fc.weight     |    300     |
|     fc.bias      |     1      |
+------------------+------------+
Total Trainable Params: 5159651


In [160]:
# https://github.com/pytorch/captum/blob/master/tutorials/models/imdb-model-cnn.pt
# Download the model from this repo and save it in models directory

model = torch.load('imdb-model-cnn.pt')
model.eval()
model = model.to(device)

/home/ubuntu/anaconda3/lib/python3.8/site-packages/torch/serialization.py:649: SourceChangeWarning: source code of class 'torch.nn.modules.sparse.Embedding' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/ubuntu/anaconda3/lib/python3.8/site-packages/torch/serialization.py:649: SourceChangeWarning: source code of class 'torch.nn.modules.container.ModuleList' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/home/ubuntu/anaconda3/lib/python3.8/site-packages/torch/serialization.py:649: SourceChangeWarning: source code of class 'torch.nn.modules.conv.Conv2d' has changed. you can retrieve the original source code by accessing 

In [161]:
def forward_with_sigmoid(input):
    return torch.sigmoid(model(input))

In [102]:
# https://ai.stanford.edu/~amaas/data/sentiment/

TEXT = torchtext.data.Field(lower=True, tokenize='spacy')
Label = torchtext.data.LabelField(dtype = torch.float)

train, test = torchtext.datasets.IMDB.splits(text_field=TEXT,
                                      label_field=Label,
                                      train='train',
                                      test='test',
                                      path='data/aclImdb')

test, _ = test.split(split_ratio = 0.04)


/home/ubuntu/anaconda3/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
/home/ubuntu/anaconda3/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: LabelField class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
/home/ubuntu/anaconda3/lib/python3.8/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired in the 0.8.0 releas

In [162]:
try_mlflow_log(mlflow.log_param, "Train Size", len(train))
try_mlflow_log(mlflow.log_param, "Test Size", len(test))

In [163]:
from torchtext import vocab

# It will automatically download Glove embedding one time

loaded_vectors = vocab.GloVe(name='6B', dim=50)

TEXT.build_vocab(train, vectors=loaded_vectors, max_size=len(loaded_vectors.stoi))    
TEXT.vocab.set_vectors(stoi=loaded_vectors.stoi, vectors=loaded_vectors.vectors, dim=loaded_vectors.dim)
Label.build_vocab(train)

print('Vocabulary Size: ', len(TEXT.vocab))

mlflow.log_param("Vocabulary Size", len(TEXT.vocab))

PAD_IND = TEXT.vocab.stoi['pad']
token_reference = TokenReferenceBase(reference_token_idx=PAD_IND)
lig = LayerIntegratedGradients(model, model.embedding)

vis_data_records_ig = []

def interpret_sentence(model, sentence, min_len = 7, label = 0):
    text = [tok.text for tok in nlp.tokenizer(sentence)]
    if len(text) < min_len:
        text += ['pad'] * (min_len - len(text))
    indexed = [TEXT.vocab.stoi[t] for t in text]

    model.zero_grad()

    input_indices = torch.tensor(indexed, device=device)
    input_indices = input_indices.unsqueeze(0)
    
    # input_indices dim: [sequence_length]
    seq_length = min_len

    # predict
    pred = forward_with_sigmoid(input_indices).item()
    pred_ind = round(pred)

    # generate reference indices for each sample
    reference_indices = token_reference.generate_reference(seq_length, device=device).unsqueeze(0)

    # compute attributions and approximation delta using layer integrated gradients
    attributions_ig, delta = lig.attribute(input_indices, reference_indices, \
                                           n_steps=500, return_convergence_delta=True)

    print('pred: ', Label.vocab.itos[pred_ind], '(', '%.2f'%pred, ')', ', delta: ', abs(delta))

    add_attributions_to_visualizer(attributions_ig, text, pred, pred_ind, label, delta, vis_data_records_ig)
    
def add_attributions_to_visualizer(attributions, text, pred, pred_ind, label, delta, vis_data_records):
    attributions = attributions.sum(dim=2).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    attributions = attributions.cpu().detach().numpy()

    # storing couple samples in an array for visualization purposes
    vis_data_records.append(visualization.VisualizationDataRecord(
                            attributions,
                            pred,
                            Label.vocab.itos[pred_ind],
                            Label.vocab.itos[label],
                            Label.vocab.itos[1],
                            attributions.sum(),       
                            text,
                            delta))


Vocabulary Size:  101513


In [164]:
interpret_sentence(model, 'It was a fantastic performance !', label=1)
interpret_sentence(model, 'Best film ever', label=1)
interpret_sentence(model, 'Such a great show!', label=1)
interpret_sentence(model, 'It was a horrible movie', label=0)
interpret_sentence(model, 'I\'ve never watched something as bad', label=0)
interpret_sentence(model, 'It is a disgusting movie!', label=0)

pred:  pos ( 0.99 ) , delta:  tensor([2.2198e-05], dtype=torch.float64)
pred:  pos ( 1.00 ) , delta:  tensor([6.6302e-05], dtype=torch.float64)
pred:  pos ( 1.00 ) , delta:  tensor([0.0003], dtype=torch.float64)
pred:  pos ( 0.69 ) , delta:  tensor([0.0003], dtype=torch.float64)
pred:  neg ( 0.22 ) , delta:  tensor([0.0011], dtype=torch.float64)
pred:  pos ( 0.80 ) , delta:  tensor([0.0008], dtype=torch.float64)


In [167]:
import pandas as pd

In [168]:
result = pd.DataFrame(columns=["raw_input", "attr_class", "attr_score", "convergence_score", "pred_prob", "true_class", "word_attributions"])

In [169]:
for i in range(0, len(vis_data_records_ig)):
    result.loc[i] = [vis_data_records_ig[i].raw_input, vis_data_records_ig[i].attr_class, vis_data_records_ig[i].attr_score, vis_data_records_ig[i].convergence_score, vis_data_records_ig[i].pred_prob, vis_data_records_ig[i].true_class, vis_data_records_ig[i].word_attributions]

In [170]:
result

,raw_input,attr_class,attr_score,convergence_score,pred_prob,true_class,word_attributions
0,"[It, was, a, fantastic, performance, !, pad]",pos,-0.608882,"[tensor(2.2198e-05, dtype=torch.float64)]",0.989221,pos,"[-0.04036963633871359, -0.26865313379737765, -..."
1,"[Best, film, ever, pad, pad, pad, pad]",pos,-1.095115,"[tensor(6.6302e-05, dtype=torch.float64)]",0.998325,pos,"[-0.22396408954050118, 0.09846007783400175, -0..."
2,"[Such, a, great, show, !, pad, pad]",pos,-0.588290,"[tensor(-0.0003, dtype=torch.float64)]",0.996762,pos,"[-0.1022866628788326, -0.6039511341403813, 0.6..."
3,"[It, was, a, horrible, movie, pad, pad]",pos,-1.817970,"[tensor(0.0003, dtype=torch.float64)]",0.688550,neg,"[-0.035739643546623116, -0.09087759821024596, ..."
4,"[I, 've, never, watched, something, as, bad]",pos,-2.448575,"[tensor(0.0011, dtype=torch.float64)]",0.216522,neg,"[-0.09314164195969875, -0.438678035818444, -0...."
5,"[It, is, a, disgusting, movie, !, pad]",pos,-2.015618,"[tensor(0.0008, dtype=torch.float64)]",0.804903,neg,"[-0.08236056046200133, -0.037739318701672284, ..."


In [171]:
word_attr_file = "word_attr.csv"
tempdir = tempfile.mkdtemp()
try:
    word_attr_file_path = os.path.join(tempdir, word_attr_file)
    result.to_csv(word_attr_file_path)

    try_mlflow_log(mlflow.log_artifact, local_path=word_attr_file_path)
finally:
    shutil.rmtree(tempdir)

In [172]:
print('Visualize attributions based on Integrated Gradients')
visualization.visualize_text(vis_data_records_ig)

Visualize attributions based on Integrated Gradients


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
pos,pos (0.99),pos,-0.61,It was a fantastic performance ! pad
pos,pos (1.00),pos,-1.10,Best film ever pad pad pad pad
pos,pos (1.00),pos,-0.59,Such a great show ! pad pad
neg,pos (0.69),pos,-1.82,It was a horrible movie pad pad
neg,neg (0.22),pos,-2.45,I 've never watched something as bad
neg,pos (0.80),pos,-2.02,It is a disgusting movie ! pad


In [173]:
for i in range(0, len(vis_data_records_ig)):
    input_string = ' '.join(vis_data_records_ig[i].raw_input)
    score = vis_data_records_ig[i].attr_score    
    try_mlflow_log(mlflow.log_metric, "Attribution Score" + str(i), float(score), step=i)
    

In [174]:
mlflow.end_run()